In [0]:
# Install Libraries
import spacy
spacy.cli.download("en_core_web_md")
!pip install rake_nltk
!pip install textacy

In [0]:
#Libraries 
from flask import Flask, request
from itertools import islice
import pandas as pd
import numpy as np
import threading
import dill as pickle
import datetime
import random 
import socket
import json
import gzip
import sys
import spacy
import tqdm
import time 
import pandas as pd
from textblob import TextBlob
from rake_nltk import Metric, Rake
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()
import textacy
import nltk
from nltk.corpus import stopwords 
from dask import dataframe as dd
import textacy.keyterms
import pickle



print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

def getFeature(row):
    FeatureDict = {}
    # Rake Topic 
    r = Rake(min_length=2, max_length=4) # Uses stopwords for english from NLTK, and all puntuation characters.
    r.extract_keywords_from_text(row)
    topics = r.get_ranked_phrases()
    
    text = nlp(row)
    # Noun Phrase 
    
    noun_phrase_pairs = []
    for chunk in text.noun_chunks:
    #   print (chunk.text)
      if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
        noun_phrase_pairs.append(chunk.text)
        
    # Sentiment 
    Sentiment = float("{0:.2f}".format(TextBlob(row).sentiment.polarity))
    
    # Negation Pairs 
    Negation_Pair = []
    negation_tokens = [tok for tok in text if tok.dep_ == 'neg']
    negation_head_tokens = [token.head for token in negation_tokens]
    for token in negation_head_tokens:
      Negation_Pair.append([token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children]])
    
    # NMF Topics 
      ## Not Now
      
    # Basic Entity Extraction 
      ## Not now 
    
    # Noun Adjective Pair ---> Rishi 
    output = []
    a_n_pair = []
    
    n_a_pair = []
    for sent in text.sents: # sentences of a doc
      # This block handles adjective noun pairs
      for token in sent: # tokens of a sentence
        if token.pos_ == "VERB" : # since verbs are central connection between nouns and adjs
#           print(token, "is a verb")
            for child in token.children: # childen of verbs
#             print('child ', child)
              if child.pos_ in ['NOUN', 'PRON','PROPN']: 
                for grand_child in child.children:
#                 print("grandchild", grand_child)
                  if grand_child.pos_ in ['ADJ']:
                    a_n_pair.append(grand_child.string.strip()+" "+child.string.strip())
      if len(a_n_pair)>0:
        output.extend(a_n_pair)
      
      # This block handles noun adj pairs
      noun_flag = False
      noun = str
      adj = str()
      adj_flag = False
      for token in sent:
        if token.pos_ == "VERB" :
#           print(token, "is a verb")
            for child in token.children:
#             print('child2 ', child)
              if noun_flag == False:
                if child.pos_ in ['NOUN', 'PRON','PROPN']:
                  noun_flag = True
                  noun = child
                  continue
              if adj_flag == False:
                if child.pos_ in ['ADJ']:
                  adj_flag = True
                  adj = child
              if noun_flag and adj_flag:
                n_a_pair.append(adj.string.strip()+" "+noun.string.strip())

      if len(n_a_pair)>0:
        output.extend(n_a_pair)
    output = list(set(output))
    
    # Subject Verb Object 
    SVOPair = list(textacy.extract.subject_verb_object_triples(text))
    # SG Rank 
    SGRankTopic = list(textacy.keyterms.sgrank(text, ngrams=(1, 2, 3, 4, 5, 6), normalize='lemma', window_width=1500, n_keyterms=10, idf=None))
    # TextRank 
    TextRank = list(textacy.keyterms.textrank(text, normalize='lemma', n_keyterms=10))
    # Key Terms Semantic Network 
    KeyTermPageRank = list(textacy.keyterms.key_terms_from_semantic_network(text, normalize='lemma', window_width=2, edge_weighting='binary', ranking_algo='pagerank', join_key_words=False, n_keyterms=5))
    
    # Topic from NMF 
    FeatureDict['RakeTopics'] = topics
    FeatureDict['NounPhrase'] = noun_phrase_pairs
    FeatureDict['Sentiment'] = Sentiment
    FeatureDict['Negation_Pair'] = Negation_Pair
    FeatureDict['SubjectVerbObject'] = SVOPair
    FeatureDict['SGRankTopic'] = SGRankTopic
    FeatureDict['TextRank'] = TextRank
    FeatureDict['PageRankTopic'] = KeyTermPageRank
    FeatureDict['NounAdjectivePair'] = output
    FeatureDict['SPacyDocObject'] = [nlp(row)]
    return FeatureDict
  
def getDocDump(row,nlp):
  NLPObject = {}
  NLPObject['Object'] = [nlp(row)]
  return NLPObject


app = Flask("GenerateRandomReviews")
@app.route('/TextFeatureDump')
def ExtractReviews():
  """
  Takes DataFrame --> NPZ to DF 
  
  """
#   DF = request.args.get('DFLocation')
  
  global nlp
  nlp = spacy.load("en_core_web_md")
#   nlp = spacy.load("en")
  data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz", lines=True)

  data = data.head(10)
  ddata = dd.from_pandas(data, npartitions=4)
#   %time ddata['DocObject']  = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getDocDump(row,nlp)))).compute(scheduler='threads')
  %time ddata['ReviewFeature'] = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getFeature(row)))).compute(scheduler='synchronous', optimize_graph='True')
  df = ddata.compute()
  
#   df.to_pickle("TestPickle.pickle")
  df.to_csv("TestAPI.csv")
#   with open("backup", 'wb') as f:
#     pickle.dump(df, f)
#   df.to_pickle("Test_Pickle.pickle")
#   dfjson = df.to_json()
#   with open('data.txt', 'w') as outfile:  
#     json.dumps(dfjson)
#   outfile.close()
  
  
#   return (df.to_json(orient='records'))
  return ("DOne")




threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':}).start()



In [0]:
#Libraries 
from flask import Flask, request
from itertools import islice
import pandas as pd
import numpy as np
import threading
import dill as pickle
import datetime
import random 
import socket
import json
import gzip
import sys
import spacy
import tqdm
import time 
import pandas as pd
from textblob import TextBlob
from rake_nltk import Metric, Rake
from textblob.np_extractors import ConllExtractor
extractor = ConllExtractor()
import textacy
import nltk
from nltk.corpus import stopwords 
from dask import dataframe as dd
import textacy.keyterms
import pickle



print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))














app = Flask("GenerateRandomReviews")
@app.route('/TextFeatureDump')
def ExtractReviews():
  """
  Takes DataFrame --> NPZ to DF 
  
  """
#   DF = request.args.get('DFLocation')
  

  
  
  
  
  global nlp
  nlp = spacy.load("en_core_web_md")
#   nlp = spacy.load("en")
  data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz", lines=True)

  data = data.head(10)
  ddata = dd.from_pandas(data, npartitions=4)
#   %time ddata['DocObject']  = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getDocDump(row,nlp)))).compute(scheduler='threads')
  %time ddata['ReviewFeature'] = ddata.map_partitions(lambda df: df.reviewText.apply((lambda row: getFeature(row)))).compute(scheduler='synchronous', optimize_graph='True')
  df = ddata.compute()
  
#   df.to_pickle("TestPickle.pickle")
  df.to_csv("TestAPI.csv")
#   with open("backup", 'wb') as f:
#     pickle.dump(df, f)
#   df.to_pickle("Test_Pickle.pickle")
#   dfjson = df.to_json()
#   with open('data.txt', 'w') as outfile:  
#     json.dumps(dfjson)
#   outfile.close()
  
  
#   return (df.to_json(orient='records'))
  return ("DOne")




threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':}).start()

